In [10]:
import os
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv


load_dotenv()

API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")

BASE_URL = "https://data.alpaca.markets/v2/stocks/bars"


symbol = "MCD"
number_of_days = 360



async def fetch_bars(session, symbol, start, end, feed):
    
    url = f"{BASE_URL}?symbols={symbol}&timeframe=5min&start={start}&end={end}&limit=10000&adjustment=raw&feed={feed}&sort=asc"
    
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": API_KEY,
        "APCA-API-SECRET-KEY": API_SECRET,
    }
    
    async with session.get(url, headers=headers) as response:
        data = await response.json()
        return data.get("bars", {}).get(symbol, [])




async def fetch_data(symbol, start_date, end_date):
    tasks = []
    async with aiohttp.ClientSession() as session:
        while start_date < end_date:
            
            next_date = start_date + timedelta(days=1)
            
            start_iso = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
            end_iso = next_date.strftime("%Y-%m-%dT%H:%M:%SZ")
            
            tasks.append(fetch_bars(session, symbol, start_iso, end_iso, "boats"))
            tasks.append(fetch_bars(session, symbol, start_iso, end_iso, "sip"))
            
            start_date = next_date
        
        results = await asyncio.gather(*tasks)
        
        
    all_bars = [bar for result in results for bar in result]
    df = pd.DataFrame(all_bars)
    
    if not df.empty:
        if "t" not in df.columns:
            print("Column 't' not found in DataFrame!")
        else:
            df.sort_values(by="t", inplace=True)
    
    return df



async def main():
    
    end_date = datetime.now(timezone.utc)
    start_date = end_date - timedelta(days=number_of_days)
    
    df = await fetch_data(symbol, start_date, end_date)
    
    if not df.empty:
        df.to_csv(f"{symbol}_{number_of_days}d_data.csv", index=False)
        print(f"Données enregistrées")
    else:
        print("Aucune donnée récupérée.")


await main()

Données enregistrées
